In [ ]:
BASE_PATH = 'drive/MyDrive/CS 410/data'

In [ ]:
!pip install -q attrs beartype polars tqdm sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "{BASE_PATH}/tmdb_5000_movies.csv.zip" -d "data"
!unzip "{BASE_PATH}/tmdb_5000_credits.csv.zip" -d "data"
!unzip "{BASE_PATH}/ratings.csv.zip" -d "data"
!unzip "{BASE_PATH}/movies.csv.zip" -d "data"

Archive:  drive/MyDrive/CS 410/data/tmdb_5000_movies.csv.zip
  inflating: data/tmdb_5000_movies.csv  
Archive:  drive/MyDrive/CS 410/data/tmdb_5000_credits.csv.zip
  inflating: data/tmdb_5000_credits.csv  
Archive:  drive/MyDrive/CS 410/data/ratings.csv.zip
  inflating: data/ratings.csv        
  inflating: data/__MACOSX/._ratings.csv  
Archive:  drive/MyDrive/CS 410/data/movies.csv.zip
  inflating: data/movies.csv         
  inflating: data/__MACOSX/._movies.csv  


In [ ]:
from IPython.display import display
from pathlib import Path
import polars as pl
import attrs
from beartype import beartype
from tqdm.auto import tqdm
import json
from functools import partial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
data_dir = Path('data')
credits_data_path = data_dir / 'tmdb_5000_credits.csv'
movies_data_path = data_dir / 'tmdb_5000_movies.csv'
ratings_data_path = data_dir / 'ratings.csv'
colab_movies_data_path = data_dir / 'movies.csv'

In [ ]:
credits_df_schema = {
    'movie_id': pl.Int64,
    'title': pl.Utf8,
    'cast': pl.Utf8,
    'crew': pl.Utf8
}

movies_df_schema = {
    'budget': pl.Int64,
    'genres': pl.Utf8,
    'homepage': pl.Utf8,
    'id': pl.Int64,
    'keywords': pl.Utf8,
    'original_language': pl.Utf8,
    'original_title': pl.Utf8,
    'overview': pl.Utf8,
    'popularity': pl.Float64,
    'production_companies': pl.Utf8,
    'production_countries': pl.Utf8,
    'release_date': pl.Datetime,
    'revenue': pl.Int64,
    'runtime': pl.Float64,
    'spoken_languages': pl.Utf8,
    'status': pl.Utf8,
    'tagline': pl.Utf8,
    'title': pl.Utf8,
    'vote_average': pl.Float64,
    'vote_count': pl.Int64,
}

credits_df = pl.read_csv(credits_data_path, schema=credits_df_schema)
credits_df = credits_df.rename({'movie_id': 'id'})
movies_df = pl.read_csv(movies_data_path, schema=movies_df_schema)
df = credits_df.join(movies_df, on='id')
df

id,title,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_right,vote_average,vote_count
i64,str,str,str,i64,str,str,str,str,str,str,f64,str,str,datetime[μs],i64,f64,str,str,str,str,f64,i64
19995,"""Avatar""","""[{""cast_id"": 242, ""character"":…","""[{""credit_id"": ""52fe4800925141…",237000000,"""[{""id"": 28, ""name"": ""Action""},…","""http://www.avatarmovie.com/""","""[{""id"": 1463, ""name"": ""culture…","""en""","""Avatar""","""In the 22nd century, a paraple…",150.437577,"""[{""name"": ""Ingenious Film Part…","""[{""iso_3166_1"": ""US"", ""name"": …",2009-12-10 00:00:00,2787965087,162.0,"""[{""iso_639_1"": ""en"", ""name"": ""…","""Released""","""Enter the World of Pandora.""","""Avatar""",7.2,11800
285,"""Pirates of the Caribbean: At W…","""[{""cast_id"": 4, ""character"": ""…","""[{""credit_id"": ""52fe4232c3a368…",300000000,"""[{""id"": 12, ""name"": ""Adventure…","""http://disney.go.com/disneypic…","""[{""id"": 270, ""name"": ""ocean""},…","""en""","""Pirates of the Caribbean: At W…","""Captain Barbossa, long believe…",139.082615,"""[{""name"": ""Walt Disney Picture…","""[{""iso_3166_1"": ""US"", ""name"": …",2007-05-19 00:00:00,961000000,169.0,"""[{""iso_639_1"": ""en"", ""name"": ""…","""Released""","""At the end of the world, the a…","""Pirates of the Caribbean: At W…",6.9,4500
206647,"""Spectre""","""[{""cast_id"": 1, ""character"": ""…","""[{""credit_id"": ""54805967c3a368…",245000000,"""[{""id"": 28, ""name"": ""Action""},…","""http://www.sonypictures.com/mo…","""[{""id"": 470, ""name"": ""spy""}, {…","""en""","""Spectre""","""A cryptic message from Bond’s …",107.376788,"""[{""name"": ""Columbia Pictures"",…","""[{""iso_3166_1"": ""GB"", ""name"": …",2015-10-26 00:00:00,880674609,148.0,"""[{""iso_639_1"": ""fr"", ""name"": ""…","""Released""","""A Plan No One Escapes""","""Spectre""",6.3,4466
49026,"""The Dark Knight Rises""","""[{""cast_id"": 2, ""character"": ""…","""[{""credit_id"": ""52fe4781c3a368…",250000000,"""[{""id"": 28, ""name"": ""Action""},…","""http://www.thedarkknightrises.…","""[{""id"": 849, ""name"": ""dc comic…","""en""","""The Dark Knight Rises""","""Following the death of Distric…",112.31295,"""[{""name"": ""Legendary Pictures""…","""[{""iso_3166_1"": ""US"", ""name"": …",2012-07-16 00:00:00,1084939099,165.0,"""[{""iso_639_1"": ""en"", ""name"": ""…","""Released""","""The Legend Ends""","""The Dark Knight Rises""",7.6,9106
49529,"""John Carter""","""[{""cast_id"": 5, ""character"": ""…","""[{""credit_id"": ""52fe479ac3a368…",260000000,"""[{""id"": 28, ""name"": ""Action""},…","""http://movies.disney.com/john-…","""[{""id"": 818, ""name"": ""based on…","""en""","""John Carter""","""John Carter is a war-weary, fo…",43.926995,"""[{""name"": ""Walt Disney Picture…","""[{""iso_3166_1"": ""US"", ""name"": …",2012-03-07 00:00:00,284139100,132.0,"""[{""iso_639_1"": ""en"", ""name"": ""…","""Released""","""Lost in our world, found in an…","""John Carter""",6.1,2124
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9367,"""El Mariachi""","""[{""cast_id"": 1, ""character"": ""…","""[{""credit_id"": ""52fe44eec3a368…",220000,"""[{""id"": 28, ""name"": ""Action""},…",null,"""[{""id"": 5616, ""name"": ""united …","""es""","""El Mariachi""","""El Mariachi just wants to play…",14.269792,"""[{""name"": ""Columbia Pictures"",…","""[{""iso_3166_1"": ""MX"", ""name"": …",1992-09-04 00:00:00,2040920,81.0,"""[{""iso_639_1"": ""es"", ""name"": ""…","""Released""","""He didn't come looking for tro…","""El Mariachi""",6.6,238
72766,"""Newlyweds""","""[{""cast_id"": 1, ""character"": ""…","""[{""credit_id"": ""52fe487dc3a368…",9000,"""[{""id"": 35, ""name"": ""Comedy""},…",null,"""[]""","""en""","""Newlyweds""","""A newlywed couple's honeymoon …",0.642552,"""[]""","""[]""",2011-12-26 00:00:00,0,85.0,"""[]""","""Released""","""A newlywed couple's honeymoon …","""Newly

In [ ]:
@beartype
@attrs.define
class Cast:
    cast_id: int
    character: str
    credit_id: str
    gender: int
    id: int
    name: str
    order: int

@beartype
@attrs.define
class Crew:
    credit_id: str
    department: str
    gender: int
    id: int
    job: str
    name: str

@beartype
@attrs.define
class Genre:
    id: int
    name: str

@beartype
@attrs.define
class Keyword:
    id: int
    name: str

@beartype
@attrs.define
class ProductionCompany:
    id: int
    name: str

def json_list_to_cls(json_str: str, cls):
    return [cls(**item) for item in json.loads(json_str)]

def get_cast(json_str):
    return [cast.name for cast in json_list_to_cls(json_str, cls=Cast)]

def get_directors(json_str):
    return [crew.name for crew in json_list_to_cls(json_str, cls=Crew) if crew.job == 'Director']

def get_genres(json_str):
    return [genre.name for genre in json_list_to_cls(json_str, cls=Genre)]

def get_keywords(json_str):
    return [keyword.name for keyword in json_list_to_cls(json_str, cls=Keyword)]

def get_production_companies(json_str):
    return [company.name for company in json_list_to_cls(json_str, cls=ProductionCompany)]

metadata_df = df.select(
    id=pl.col('id'),
    title=pl.col('title'),
    overview=pl.col('overview'),
    tagline=pl.col('tagline'),
    release_year=pl.col('release_date').dt.year(),
    cast=pl.col('cast').map_elements(get_cast, return_dtype=pl.List(pl.Utf8)).list.head(3),
    directors=pl.col('crew').map_elements(get_directors, return_dtype=pl.List(pl.Utf8)).list.head(3),
    genres=pl.col('genres').map_elements(get_genres, return_dtype=pl.List(pl.Utf8)),
    keywords=pl.col('keywords').map_elements(get_keywords, return_dtype=pl.List(pl.Utf8)),
    production_companies=pl.col('production_companies').map_elements(get_production_companies, return_dtype=pl.List(pl.Utf8)).list.head(3),
)

In [ ]:
description_df = metadata_df.select(
    id=pl.col('id'),
    title=pl.col('title'),
    description=pl.concat_str(
        [
            'Title: ' + pl.col('title'),
            ('Genres: ' + pl.col('genres').list.join(', ')).fill_null(''),
            ('Keywords: ' + pl.col('keywords').list.join(', ')).fill_null(''),
            ('Tagline: ' + pl.col('tagline')).fill_null(''),
            ('Overview: ' + pl.col('overview')).fill_null(''),
            ('Cast: ' + pl.col('cast').list.join(', ')).fill_null(''),
            ('Directors: ' + pl.col('directors').list.join(', ')).fill_null(''),
            ('Production companies: ' + pl.col('production_companies').list.join(', ')).fill_null(''),
            ('Release year: ' + pl.col('release_year').cast(pl.Utf8)).fill_null(''),
        ],
        separator='. '
    )
)
description_df

id,title,description
i64,str,str
19995,"""Avatar""","""Title: Avatar. Genres: Action,…"
285,"""Pirates of the Caribbean: At W…","""Title: Pirates of the Caribbea…"
206647,"""Spectre""","""Title: Spectre. Genres: Action…"
49026,"""The Dark Knight Rises""","""Title: The Dark Knight Rises. …"
49529,"""John Carter""","""Title: John Carter. Genres: Ac…"
…,…,…
9367,"""El Mariachi""","""Title: El Mariachi. Genres: Ac…"
72766,"""Newlyweds""","""Title: Newlyweds. Genres: Come…"
231617,"""Signed, Sealed, Delivered""","""Title: Signed, Sealed, Deliver…"


In [ ]:
tfidf_df = description_df.with_columns(description=pl.col('description').str.to_lowercase())
title_tfidf = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    analyzer='word',
    # use_idf=True,
    use_idf=False,
    norm='l2',
    smooth_idf=True,
    sublinear_tf=True,
)
title_tfidf_matrix = title_tfidf.fit_transform(tfidf_df['title'])
title_cosine_sim = linear_kernel(title_tfidf_matrix, title_tfidf_matrix)

metadata_tfidf = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    analyzer='word',
    use_idf=True,
    # use_idf=False,
    norm='l2',
    smooth_idf=True,
    sublinear_tf=True,
)
metadata_tfidf_matrix = metadata_tfidf.fit_transform(tfidf_df['description'])
metadata_cosine_sim = linear_kernel(metadata_tfidf_matrix, metadata_tfidf_matrix)

In [ ]:
def get_tfidf_recommendations(query):
    query_title_tfidf = title_tfidf.transform([query])
    query_metadata_tfidf = metadata_tfidf.transform([query])

    cosine_sim_title = linear_kernel(query_title_tfidf, title_tfidf_matrix)
    cosine_sim_metadata = linear_kernel(query_metadata_tfidf, metadata_tfidf_matrix)

    cosine_sim = cosine_sim_title * 0.5 + cosine_sim_metadata * 0.5
    indices = cosine_sim.argsort()[0][::-1]
    return df[indices][['title', 'overview']]

get_tfidf_recommendations('pirates of the caribbean')

title,overview
str,str
"""Pirates of the Caribbean: At W…","""Captain Barbossa, long believe…"
"""Pirates of the Caribbean: On S…","""Captain Jack Sparrow crosses p…"
"""Pirates of the Caribbean: The …","""Jack Sparrow, a freewheeling 1…"
"""Pirates of the Caribbean: Dead…","""Captain Jack Sparrow works his…"
"""VeggieTales: The Pirates Who D…","""Set Sail For Adventure! A boat…"
…,…
"""Veronika Decides to Die""","""After a frantic suicide attemp…"
"""Crocodile Dundee""","""When a New York reporter pluck…"
"""Ultramarines: A Warhammer 40,0…","""A squad of Ultramarines answer…"


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(description_df['description'].to_list())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_st_recommendations(query):
    query_embedding = model.encode(query)
    similarity = model.similarity(query_embedding, embeddings)[0].numpy()
    indices = similarity.argsort()[::-1]
    return df[indices][['title', 'overview']]

get_st_recommendations('pirates of the caribbean')

title,overview
str,str
"""Pirates of the Caribbean: The …","""Jack Sparrow, a freewheeling 1…"
"""Pirates of the Caribbean: Dead…","""Captain Jack Sparrow works his…"
"""Pirates of the Caribbean: At W…","""Captain Barbossa, long believe…"
"""Pirates of the Caribbean: On S…","""Captain Jack Sparrow crosses p…"
"""The Pirate""","""A girl is engaged to the local…"
…,…
"""Stick It""","""Haley is a naturally gifted at…"
"""Semi-Pro""","""Jackie Moon is the owner, prom…"
"""Get on the Bus""","""Get On the Bus follows several…"


In [ ]:
display(get_tfidf_recommendations('movies about a pirate named jack'))
display(get_st_recommendations('movies about a pirate named jack'))

title,overview
str,str
"""The Pirate""","""A girl is engaged to the local…"
"""Jack and Jill""","""Jack Sadelstein, a successful …"
"""Jack Reacher""","""In an innocent heartland city,…"
"""Casino Jack""","""Based on a true story, a hot s…"
"""Space Pirate Captain Harlock""","""Space Pirate Captain Harlock a…"
…,…
"""The Texas Chain Saw Massacre""","""Five friends visiting their gr…"
"""The Runaways""","""Joan Jett and Cherie Currie, t…"
"""Fiddler on the Roof""","""This lavishly produced and cri…"


title,overview
str,str
"""Pirates of the Caribbean: On S…","""Captain Jack Sparrow crosses p…"
"""Pirates of the Caribbean: The …","""Jack Sparrow, a freewheeling 1…"
"""Pirates of the Caribbean: Dead…","""Captain Jack Sparrow works his…"
"""The Pirate""","""A girl is engaged to the local…"
"""The Pirates! In an Adventure w…","""The luxuriantly bearded Pirate…"
…,…
"""The Help""","""Aibileen Clark is a middle-age…"
"""Mona Lisa Smile""","""The story of Katherine Ann Wat…"
"""Semi-Pro""","""Jackie Moon is the owner, prom…"


In [ ]:
display(get_tfidf_recommendations('james cameron movies'))
display(get_st_recommendations('james cameron movies'))

title,overview
str,str
"""The Assassination of Jesse Jam…","""Outlaw Jesse James is rumored …"
"""Harriet the Spy""","""When the secret notebook of a …"
"""Grindhouse""","""Two full length feature horror…"
"""Abandoned""","""When their yacht capsizes duri…"
"""Ed Wood""","""The mostly true story of the l…"
…,…
"""Return to the Blue Lagoon""","""In this sequel to the 1980 cla…"
"""Inside Llewyn Davis""","""In Greenwich Village in the ea…"
"""The Perez Family""","""In the midst of the Mariel boa…"


title,overview
str,str
"""Knight and Day""","""A fugitive couple goes on a gl…"
"""He's Just Not That Into You""","""Remember that really cute guy …"
"""Trainspotting""","""Renton, deeply immersed in the…"
"""Inside Job""","""A film that exposes the shocki…"
"""Sex Tape""","""When Jay and Annie first got t…"
…,…
"""Legends of Oz: Dorothy's Retur…","""Dorothy wakes up in post-torna…"
"""Western Religion""","""The year is 1879. Gunfighters …"
"""The Clan of the Cave Bear""","""Natural changes have the clans…"


# Content Based Filtering Evaluation


In [ ]:
def get_model_results(model, query, top_k=5):
  if model == 'tf-idf':
    results = get_tfidf_recommendations(query)
  elif model == 'sentence-transformer':
    results = get_st_recommendations(query)
  return results.head(top_k)['title'].to_list()

In [ ]:
#Given the information about the metadata being considered, ChatGPT's ground truth results for the query "pirates of the caribbean" is the following
true_results = ["Pirates of the Caribbean: The Curse of the Black Pearl", "Pirates of the Caribbean: Dead Man's Chest", "Pirates of the Caribbean: At World's End", "Pirates of the Caribbean: On Stranger Tides", "Pirates of the Caribbean: Dead Men Tell No Tales"]

In [ ]:
def calculate_map(query, true_results, model, top_k=5):
  model_results = get_model_results(model, query, top_k)
  print(f"{model} results: {model_results}")
  average_precision = 0
  relevant_count = 0
  for i in range(top_k):
    if model_results[i] in true_results:
      relevant_count += 1
      average_precision += relevant_count / (i + 1)

  if relevant_count > 0:
    average_precision /= relevant_count

  return average_precision

def compare_map(query, true_results, top_k):
  tf_idf_map = calculate_map(query, true_results, 'tf-idf', top_k)
  st_map = calculate_map(query, true_results, 'sentence-transformer', top_k)
  print(f"Tf-idf MAP: {tf_idf_map}")
  print(f"Sentence transformer MAP: {st_map}")
  if tf_idf_map == st_map:
    return "Same MAP values"
  elif tf_idf_map > st_map:
    return "TF-IDF MAP is better"
  else:
    return "Sentence-transformer MAP is better"

In [ ]:
query = 'pirates of the caribbean'
compare_map(query, true_results, 5)

tf-idf results: ["Pirates of the Caribbean: At World's End", 'Pirates of the Caribbean: On Stranger Tides', 'Pirates of the Caribbean: The Curse of the Black Pearl', "Pirates of the Caribbean: Dead Man's Chest", "VeggieTales: The Pirates Who Don't Do Anything"]
sentence-transformer results: ['Pirates of the Caribbean: The Curse of the Black Pearl', "Pirates of the Caribbean: Dead Man's Chest", "Pirates of the Caribbean: At World's End", 'Pirates of the Caribbean: On Stranger Tides', 'The Pirate']
Tf-idf MAP: 1.0
Sentence transformer MAP: 1.0


'Same MAP values'

In [ ]:
query = 'movies about a pirate named jack'
true_results = ["Pirates of the Caribbean: The Curse of the Black Pearl", "Pirates of the Caribbean: Dead Man's Chest", "Pirates of the Caribbean: At World's End", "Pirates of the Caribbean: On Stranger Tides", "Pirates of the Caribbean: Dead Men Tell No Tales"]
compare_map(query, true_results, 5)

tf-idf results: ['The Pirate', 'Jack and Jill', 'Jack Reacher', 'Casino Jack', 'Space Pirate Captain Harlock']
sentence-transformer results: ['Pirates of the Caribbean: On Stranger Tides', 'Pirates of the Caribbean: The Curse of the Black Pearl', "Pirates of the Caribbean: Dead Man's Chest", 'The Pirate', 'The Pirates! In an Adventure with Scientists!']
Tf-idf MAP: 0
Sentence transformer MAP: 1.0


'Sentence-transformer MAP is better'

In [ ]:
query = 'action movies'
true_results = [
    "Die Hard",
    "Mad Max: Fury Road",
    "The Dark Knight",
    "John Wick",
    "Gladiator",
    "The Matrix",
    "Inception",
    "Terminator 2: Judgment Day",
    "The Avengers",
    "Mission: Impossible – Fallout"
]
top_k = 10
compare_map(query, true_results, top_k)

tf-idf results: ['Last Action Hero', 'Action Jackson', 'A Civil Action', 'Looney Tunes: Back in Action', 'Grindhouse', 'Abandoned', 'Ed Wood', 'Blow Out', 'Rugrats Go Wild', 'Snow Day']
sentence-transformer results: ['Action Jackson', 'Last Action Hero', 'Taken 3', 'Redbelt', 'RED', 'Drive', 'Basic', 'Bad Company', 'The Last Shot', 'Die Hard: With a Vengeance']
Tf-idf MAP: 0
Sentence transformer MAP: 0


'Same MAP values'

In [ ]:
query = 'action genre movies'
#true results stayed the same
compare_map(query, true_results, top_k)

tf-idf results: ['Last Action Hero', 'Action Jackson', 'A Civil Action', 'Looney Tunes: Back in Action', 'The Walking Deceased', 'Grindhouse', 'Mutual Appreciation', 'Superhero Movie', 'Cinderella', 'Abandoned']
sentence-transformer results: ['Action Jackson', 'Last Action Hero', 'Basic', 'Taken 3', 'Heavy Metal', 'RED', 'Bad Company', 'The Helix... Loaded', 'The Canyons', 'Knight and Day']
Tf-idf MAP: 0
Sentence transformer MAP: 0


'Same MAP values'

In [ ]:
query = 'movies about super heroes'
true_results = [
    "The Dark Knight",
    "Avengers: Endgame",
    "Spider-Man: No Way Home",
    "Iron Man",
    "Black Panther",
    "Superman: The Movie",
    "Guardians of the Galaxy",
    "Wonder Woman",
    "Deadpool",
    "X-Men: Days of Future Past"
]
compare_map(query, true_results, top_k)

tf-idf results: ['Super 8', 'Super', 'Heroes of Dirt', 'Imaginary Heroes', 'Super Hybrid', 'The Blood of Heroes', 'Super Troopers', 'Super Size Me', 'My Super Ex-Girlfriend', 'Super Mario Bros.']
sentence-transformer results: ['Batman', 'All Superheroes Must Die', 'Mystery Men', 'American Hero', 'Superhero Movie', 'Watchmen', 'Batman v Superman: Dawn of Justice', 'Superman III', 'Superman IV: The Quest for Peace', 'Superman II']
Tf-idf MAP: 0
Sentence transformer MAP: 0


'Same MAP values'

In [ ]:
query = 'spider man'
spider_man_movies = [
    "Spider-Man: No Way Home",
    "Spider-Man: Into the Spider-Verse",
    "Spider-Man",
    "Spider-Man 2",
    "Spider-Man: Far From Home",
    "Spider-Man 3",
    "The Amazing Spider-Man",
    "The Amazing Spider-Man 2",
    "Spider-Man: Homecoming",
    "Spider-Man: Across the Spider-Verse"
]
compare_map(query, spider_man_movies, top_k)

tf-idf results: ['Spider-Man 3', 'Spider-Man', 'Spider-Man 2', 'The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Spider', 'Nothing But a Man', 'The Man', 'A Serious Man', 'Man on Fire']
sentence-transformer results: ['Spider-Man', 'Spider-Man 3', 'Spider-Man 2', 'The Amazing Spider-Man 2', 'The Amazing Spider-Man', 'Arachnophobia', 'Spider', 'Kingdom of the Spiders', 'Along Came a Spider', 'Superhero Movie']
Tf-idf MAP: 1.0
Sentence transformer MAP: 1.0


'Same MAP values'

In [ ]:
query = "movies about a boy named peter parker"
compare_map(query, spider_man_movies, top_k)

tf-idf results: ['Parker', 'Nowhere Boy', 'The Boy', "That's My Boy", 'About a Boy', 'Little Boy', "Grandma's Boy", 'Baby Boy', 'Astro Boy', 'Good Boy!']
sentence-transformer results: ['Spider-Man', 'The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Spider-Man 2', 'Hook', 'Boyhood', 'Little Boy', "That's My Boy", 'Due Date', 'The Mudge Boy']
Tf-idf MAP: 0
Sentence transformer MAP: 1.0


'Sentence-transformer MAP is better'

# Collaborative Filtering

https://www.geeksforgeeks.org/recommendation-system-in-python/

In [ ]:
ratings = pd.read_csv(ratings_data_path)
ratings.head()

,userId,movieId,rating,timestamp
0,877,4155,5,1651201566
1,305,7661,2,1639553712
2,381,8423,2,1610704432
3,208,6433,1,1650223767
4,47,7752,4,1663998365


In [ ]:
movies = pd.read_csv(colab_movies_data_path)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# user rating frequency
# counts the number of movies each user has rated.
user_freq = ratings[['userId', 'movieId']].groupby(
    'userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
print(user_freq.head())

   userId  n_ratings
0       1        120
1       2        105
2       3         89
3       4        100
4       5        107


In [ ]:
# get the mean rating of each movie
mean_rating = ratings.groupby('movieId')[['rating']].mean()
# index of lowest mean rated movie
lowest_rated = mean_rating['rating'].idxmin()
# index of highest mean rated movie
highest_rated = mean_rating['rating'].idxmax()

In [ ]:
# all the ratings of the highest rated movie
ratings[ratings['movieId']==highest_rated]

,userId,movieId,rating,timestamp
10060,171,7831,5,1632880028
51716,992,7831,4,1646316676
52680,120,7831,5,1653456682
84994,610,7831,5,1632111958
91240,540,7831,5,1619124112


In [ ]:
# all the ratings of the lowest rated movie
ratings[ratings['movieId']==lowest_rated]

,userId,movieId,rating,timestamp
78025,667,1285,1,1613097939
78504,840,1285,1,1643070897
84758,979,1285,1,1616319498


In [ ]:
from scipy.sparse import csr_matrix

def create_matrix(df):

    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())

    # key: userID/movieID, value: index of sparse matrix
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
    print(user_mapper)

    # key: index of sparse matrix, value: userID/movieID
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    print(user_inv_mapper)

    # user_index = [user_mapper[i] for i in df['userId']]
    user_index = []
    for i in df['userId']:
        print("i: ", i)
        # print("user mapper: ", user_mapper[i])
        user_index.append(user_mapper[i])

    movie_index = [movie_mapper[i] for i in df['movieId']]
    print(user_index)

    print(df["rating"])
    # csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)]) where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    print(X)

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [ ]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

Streaming output truncated to the last 5000 lines.
i:  276
i:  850
i:  550
i:  624
i:  330
i:  111
i:  826
i:  73
i:  819
i:  370
i:  603
i:  945
i:  855
i:  680
i:  573
i:  852
i:  616
i:  484
i:  818
i:  24
i:  636
i:  897
i:  801
i:  857
i:  121
i:  554
i:  277
i:  507
i:  789
i:  588
i:  295
i:  714
i:  356
i:  66
i:  989
i:  217
i:  597
i:  810
i:  745
i:  253
i:  802
i:  274
i:  471
i:  50
i:  31
i:  923
i:  27
i:  818
i:  816
i:  677
i:  281
i:  479
i:  675
i:  411
i:  619
i:  788
i:  921
i:  521
i:  496
i:  223
i:  878
i:  137
i:  990
i:  529
i:  611
i:  572
i:  415
i:  212
i:  548
i:  984
i:  874
i:  668
i:  831
i:  225
i:  486
i:  553
i:  290
i:  144
i:  419
i:  907
i:  601
i:  446
i:  85
i:  123
i:  582
i:  176
i:  114
i:  704
i:  142
i:  490
i:  318
i:  861
i:  389
i:  70
i:  890
i:  147
i:  765
i:  255
i:  212
i:  893
i:  282
i:  249
i:  861
i:  957
i:  364
i:  615
i:  315
i:  859
i:  317
i:  232
i:  252
i:  499
i:  747
i:  946
i:  134
i:  25
i:  701
i:  720
i:  225
i:  35

In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []

    if movie_id not in movie_mapper:
        print(f"Movie ID {movie_id} not found in movie_mapper!")
        return []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)

    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])

    neighbour_ids.pop(0)
    return neighbour_ids


In [ ]:
def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    df1 = ratings[ratings['userId'] == user_id]

    movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]
    print(f"Movie ID: {movie_id}")

    movie_titles = dict(zip(movies['movieId'], movies['title']))

    similar_ids = find_similar_movies(movie_id, X, k)
    print(similar_ids)

    print(f"Since you watched {movie_titles[movie_id]}, you might also like:")

    for i in similar_ids:
        if i in movie_titles:
            print(movie_titles[i])


In [ ]:
user_id = 150
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)

Movie ID: 1245
[np.int64(6189), np.int64(3115), np.int64(6291), np.int64(7702), np.int64(1748), np.int64(6196), np.int64(1154), np.int64(7167), np.int64(1311), np.int64(754)]
Since you watched Miller's Crossing (1990), you might also like:
Flawless (1999)
Lilya 4-Ever (Lilja 4-ever) (2002)
Bells of St. Mary's, The (1945)
Dark City (1998)
Cradle 2 the Grave (2003)
Japanese Story (2003)
